## 🏅 Olympic Medal Winner Prediction

Given *data about Olympic athletes*, let's try to predict whether a given competitor will be a **medal winner** or not.

We will use a TensorFlow ANN to make our predictions.

Data source: https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix